In [115]:
import pandas as pd
import numpy as np

In [133]:
n400 = pd.read_csv("N400_by_trial.csv")
p600 = pd.read_csv("P600_by_trial.csv")
spr = pd.read_csv("SPR_by_trial.csv")

In [169]:
def extract_features(dataframe: pd.DataFrame, condition):
    """
    Extracts features from a dataframe of ERPs / SRP's. 
    
    Args:
        dataframe: A dataframe of ERPs / SRP's.
        condition: The condition to extract features from.
    Returns:
        A numpy array of features.
    """
    
    features = dataframe[dataframe["Condition"] == condition]
    features = features.drop(["Condition"], axis=1)
    features = features.set_index("ItemNum")
    features = np.expand_dims(features.to_numpy(), axis=0)
    return features

n400_control = extract_features(n400, "control")
n400_script_related = extract_features(n400, "script-related")
n400_script_unrelated = extract_features(n400, "script-unrelated")

p600_control = extract_features(p600, "control")
p600_script_related = extract_features(p600, "script-related")
p600_script_unrelated = extract_features(p600, "script-unrelated")

spr_control = extract_features(spr, "control")
spr_script_related = extract_features(spr, "script-related")
spr_script_unrelated = extract_features(spr, "script-unrelated")

### Visualise data

In [175]:
def print_item(item = 0):
    print(f"ItemNum: {item+1}")
    print(f"ERP's (n400): \n{n400_control[0][item]}")
    print(f"SPR: {spr_control[0][item]}")

print_item(0)

ItemNum: 1
ERP's (n400): 
[-1.79205279 -1.86712636 -1.70394696 -3.9231718  -4.74066061 -4.90966067
 -2.37030723 -1.98819744 -3.95095261 -3.93343971 -4.15195217 -2.20787836
 -3.72650287 -2.34174259  0.50864363 -1.46534323 -2.34354867 -2.66679243
 -0.31249529  0.80925981 -1.85934109 -1.26222229 -1.31437606 -1.49421967
 -0.46220971 -0.38511491]
SPR: [432.36363636]


In [200]:
# check data shape
print(spr_control.shape)
print(n400_control.shape)
print(p600_control.shape)

# check data type
print(type(spr_control))
print(n400_control.dtype)
print(p600_control.dtype)

(1, 90, 1)
(1, 90, 26)
(1, 90, 26)
<class 'numpy.ndarray'>
float64
float64


### Prepare data for training

In [153]:
from sklearn.model_selection import train_test_split

In [220]:
X = np.concatenate((n400_control, p600_control), axis=0)
y = np.reshape(spr_control, (90, 1, 1))

# reshape x to have 90 in first dimension
X = np.reshape(X, (90, X.shape[0], 26))

In [221]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [222]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

X_train shape: (72, 2, 26)
y_train shape: (72, 1, 1)


### Construct RNN Model

In [99]:
# from tensorflow import Sequential
from keras import layers, Sequential

In [280]:
# make an rnn model that takes in the n400 and p600 data and predicts the spr data

model = Sequential()
model.add(layers.Input(shape=(2, 26)))
model.add(layers.LSTM(128, input_shape=(2, 26)))
model.add(layers.Reshape((1, 128)))
model.add(layers.LSTM(128, input_shape=(1, 128)))
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dense(16, activation="relu"))
model.add(layers.Dense(1, activation="relu"))

model.compile(optimizer="adam", loss="mse", metrics=['mae', 'mse'])

model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_31 (LSTM)              (None, 128)               79360     
                                                                 
 reshape_6 (Reshape)         (None, 1, 128)            0         
                                                                 
 lstm_32 (LSTM)              (None, 128)               131584    
                                                                 
 dense_57 (Dense)            (None, 64)                8256      
                                                                 
 dense_58 (Dense)            (None, 32)                2080      
                                                                 
 dense_59 (Dense)            (None, 16)                528       
                                                                 
 dense_60 (Dense)            (None, 1)               

In [284]:
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
3/3 [==============================] - 0s 15ms/step - loss: 2129.7720 - mae: 24.0836 - mse: 2129.7720
Epoch 2/100
3/3 [==============================] - 0s 13ms/step - loss: 1890.0201 - mae: 21.6467 - mse: 1890.0201
Epoch 3/100
3/3 [==============================] - 0s 15ms/step - loss: 1767.5062 - mae: 21.0714 - mse: 1767.5062
Epoch 4/100
3/3 [==============================] - 0s 14ms/step - loss: 1625.7152 - mae: 19.9435 - mse: 1625.7152
Epoch 5/100
3/3 [==============================] - 0s 15ms/step - loss: 1626.9552 - mae: 19.2155 - mse: 1626.9552
Epoch 6/100
3/3 [==============================] - 0s 15ms/step - loss: 1513.9044 - mae: 19.4648 - mse: 1513.9044
Epoch 7/100
3/3 [==============================] - 0s 15ms/step - loss: 1432.0804 - mae: 19.2577 - mse: 1432.0804
Epoch 8/100
3/3 [==============================] - 0s 14ms/step - loss: 1372.0643 - mae: 18.3278 - mse: 1372.0643
Epoch 9/100
3/3 [==============================] - 0s 17ms/step - loss: 1272.5100 - mae:

### Test model

In [244]:
from sklearn.metrics import mean_squared_error

In [285]:
pred = model.predict(X_test)

# remove 1 dim from y_test to match pred
y_test_ = np.squeeze(y_test, axis=1)
pred = np.squeeze(pred, axis=1)

mse = mean_squared_error(y_test_, pred)
print(f"\nMSE: {mse}")

print("\nFirst 5 predictions: ")
for i in range(min(len(pred), 5)):
    print(f"pred: {pred[i]}", end=" ")
    print(f"actual: {y_test_[i]}")

1/1 [==============================] - 0s 56ms/step

MSE: 24788.251399538858

First 5 predictions: 
pred: 623.7996215820312 actual: 594.9
pred: 387.26104736328125 actual: 427.8
pred: 646.1466674804688 actual: 425.4
pred: 342.8466796875 actual: 471.9
pred: 463.86016845703125 actual: 432.3636363636364
